# Laboratorio 4 - Parte 2

In [1]:
import pefile
import os
import re
import magic
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("VirusSample.csv")

## Exploracion de datos

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9795 entries, 0 to 9794
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   file    9795 non-null   object
 1   api     9795 non-null   object
 2   class   9795 non-null   object
dtypes: object(3)
memory usage: 229.7+ KB


In [4]:
df.head()

,file,api,class
0,7ff49f2f0912352416b05c010f35f402cc79feed,"IntersectRect,GetCurrentProcess,GetVersion",Virus
1,50cc6c99ec285d0db45dde07d8fdc18d9098c5b6,"GetCaretBlinkTime,CountClipboardFormats,GetCon...",Virus
2,f77c6bd4aebacd1a01d02e0cb20642ebf2d32929,"VarR8Pow,GetClipboardViewer,GetInputDesktop,Ge...",Virus
3,349c367c5b88fbb6cafae5d7109588d7250e16b5,"SetTraceCallback,CopyAcceleratorTableW,GetProc...",Virus
4,021f4aa86b520e1d606ab26699c35546bcd00c27,"SHLoadNonloadedIconOverlayIdentifiers,VarUI8Fr...",Virus


In [5]:
#Tokenizar las llamadas del api
#Bigramas o Trigramas

df["class"].value_counts()

Trojan        6153
Virus         2367
Backdoor       447
Worms          441
Adware         222
Agent          102
Downloader      31
Spyware         11
Ransomware      10
Riskware         4
Dropper          4
Crypt            2
Keylogger        1
Name: class, dtype: int64

## Preprocesamiento de datos

### Eliminacion de datos obsoletos

In [6]:
df.drop(df[df["class"] == "Keylogger"].index, inplace=True)
df.drop(df[df["class"] == "Crypt"].index, inplace=True)
df.drop(df[df["class"] == "Dropper"].index, inplace=True)
df.drop(df[df["class"] == "Riskware"].index, inplace=True)
df.drop(df[df["class"] == "Ransomware"].index, inplace=True)
df.drop(df[df["class"] == "Spyware"].index, inplace=True)
df.drop(df[df["class"] == "Downloader"].index, inplace=True)

In [7]:
df["class"].value_counts()

Trojan      6153
Virus       2367
Backdoor     447
Worms        441
Adware       222
Agent        102
Name: class, dtype: int64

### Label encoder para dar valor hacia las llamadas del API

In [8]:
from sklearn.preprocessing import LabelEncoder
labelEncoder = LabelEncoder()
df["class"]= labelEncoder.fit_transform(df["class"])

In [9]:
#Elimando ya las clases que no iban a afectar al modelo, se hace un label encoder para obtener datos
target = df["class"]
df.drop(columns=["class"], inplace=True)
feautures = df

In [11]:
features = pd.DataFrame(df, dtype=str)
features.apply(labelEncoder.fit_transform)

feautures = feautures.apply(labelEncoder.fit_transform).values[:,:]
feautures

array([[4887, 1240],
       [3104,  599],
       [9414, 2501],
       ...,
       [8390, 2708],
       [7864, 1209],
       [7221, 2299]])

### Separando datos para los modelos

In [14]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(feautures, target, test_size=0.3, random_state=42)

In [15]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(6812, 2)
(2920, 2)
(6812,)
(2920,)


In [39]:
target.value_counts()

3    6153
4    2367
2     447
5     441
0     222
1     102
Name: class, dtype: int64

Observamos que la jerarquia que creo el label encoder fue esta manera:
- 0 --> Adware
- 1 --> Agent
- 2 --> Backdoor
- 3 --> Trojan
- 4 --> Virus
- 5 --> Worms

### Escalando datos necesarios 

In [17]:
from sklearn.preprocessing import MinMaxScaler
escalatedData = x_train  #Features (llamadas del api)
scaler = MinMaxScaler()
print(scaler.fit(escalatedData))

MinMaxScaler()


In [18]:
print(scaler.transform(escalatedData))

[[0.10102775 0.33616876]
 [0.9684481  0.4954066 ]
 [0.63556012 0.15651582]
 ...
 [0.23535457 0.44538959]
 [0.96495375 0.49880912]
 [0.65765673 0.31201089]]


In [19]:
x_train = scaler.transform(escalatedData)
x_test = scaler.transform(x_test)

## Modelos de clasificacion

In [35]:
from sklearn.model_selection import cross_val_score
from sklearn import metrics, model_selection, tree
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold

### Naive Bayes

In [60]:
bayes = GaussianNB()
bayes.fit(x_train, y_train)

GaussianNB()

In [61]:
accuracy_training = bayes.score(x_train,y_train)
accuracy_test = bayes.score(x_test,y_test)

print("Accuracy of traning: {0}".format(accuracy_training) )
print("Accuracy of testing: {0}".format(accuracy_test) )

Accuracy of traning: 0.608191426893717
Accuracy of testing: 0.6102739726027397


- 0 --> Adware
- 1 --> Agent
- 2 --> Backdoor
- 3 --> Trojan
- 4 --> Virus
- 5 --> Worms

In [62]:
y_predict = bayes.predict(x_test)
print(classification_report(y_test,y_predict))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        58
           1       0.00      0.00      0.00        30
           2       0.00      0.00      0.00       149
           3       0.63      0.95      0.76      1849
           4       0.16      0.03      0.05       710
           5       0.00      0.00      0.00       124

    accuracy                           0.61      2920
   macro avg       0.13      0.16      0.13      2920
weighted avg       0.44      0.61      0.49      2920



/home/mario/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mario/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mario/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Cross Validation

In [63]:
bayes_cv_score = cross_val_score(estimator=bayes, X=x_train, y=y_train, cv=12, n_jobs=-1)
print(bayes_cv_score)

[0.58098592 0.60387324 0.61795775 0.60387324 0.63028169 0.59683099
 0.60387324 0.63204225 0.5978836  0.61022928 0.61022928 0.62786596]


#### Kfolds

In [64]:
cv = KFold(n_splits=10, random_state=1, shuffle=True)
scores = cross_val_score(bayes,x_train,y_train, scoring='accuracy', cv=cv, n_jobs=-1)
scores

array([0.60997067, 0.60117302, 0.62114537, 0.6020558 , 0.62848752,
       0.62114537, 0.61380323, 0.59324523, 0.6123348 , 0.6123348 ])

In [65]:
print("Accuracy: {0} ".format(np.mean(scores)))

Accuracy: 0.6115695824236396 


Podemos observar de que nuestro modelo contiene un accuracy promedio de 0.612. Ademas, podemos observar de que el trojano fue el dato con mayor precision, recall y fi-score, dejando atras los demas debido a que sus metricas se quedaban en 0 menor de 20%

### Decision Tree

In [49]:
clf = tree.DecisionTreeClassifier()
clf.fit(x_train,y_train)

DecisionTreeClassifier()

In [50]:
accuracy_training = clf.score(x_train,y_train)
accuracy_test = clf.score(x_test,y_test)

print("Accuracy of traning: {0}".format(accuracy_training) )
print("Accuracy of testing: {0}".format(accuracy_test) )

Accuracy of traning: 1.0
Accuracy of testing: 0.8780821917808219


- 0 --> Adware
- 1 --> Agent
- 2 --> Backdoor
- 3 --> Trojan
- 4 --> Virus
- 5 --> Worms

In [51]:
y_predict = clf.predict(x_test)
print(classification_report(y_test,y_predict))

              precision    recall  f1-score   support

           0       0.58      0.62      0.60        58
           1       0.22      0.17      0.19        30
           2       0.89      0.88      0.88       149
           3       0.91      0.92      0.91      1849
           4       0.89      0.86      0.87       710
           5       0.65      0.69      0.67       124

    accuracy                           0.88      2920
   macro avg       0.69      0.69      0.69      2920
weighted avg       0.88      0.88      0.88      2920



#### Cross Validation

In [52]:
tree_cv_score = cross_val_score(estimator=bayes, X=x_train, y=y_train, cv=12, n_jobs=-1)
print(tree_cv_score)

[0.58098592 0.60387324 0.61795775 0.60387324 0.63028169 0.59683099
 0.60387324 0.63204225 0.5978836  0.61022928 0.61022928 0.62786596]


#### Kfolds

In [53]:
cv = KFold(n_splits=10, random_state=1, shuffle=True)
scores = cross_val_score(clf,x_train,y_train, scoring='accuracy', cv=cv, n_jobs=-1)
scores

array([0.88856305, 0.87536657, 0.88105727, 0.88839941, 0.87812041,
       0.88399413, 0.88105727, 0.87518355, 0.8928047 , 0.85903084])

In [57]:
print("Accuracy: {0}".format(np.mean(scores)))

Accuracy: 0.8803577195860839


Podemos observar de que nuestro segundo modele contiene un accuracy promedio de 0.88. Ademas, podemos observar de que el trojano nuevamente fue el dato con mayor precision, recall y fi-score. Una clave diferencia del primer modelo es que tenemos metricas altas para los demas malware, dandole mayor valor a utilizar un decision tree en vez de Naive Bayes

## Conclusiones

#### ¿Se lograron obtener mejores métricas que las obtenidas en el artículo para la clasificación de malware?

En este caso, observamos que ambos modelos que escogimos, siendo Decision tree y Naive Bayes, no lograron obtener un resultado mas alto que el articulo. Tomando en cuenta que el label encoder logro organizar las clasificaciones de nuestro target, la escalacion vario los valores de manera que impactara la precision de los modelos. Otra manera que se pudiera haber aumentado las metricas es haber utilizado otra separacion de datos para un valid set, con el fin de realizar pruebas y revisar nuevamente el preprocesamiento.